# 4. Logic-Geometric Programming: Skeleton Interface

* Formally, in papers, a skeleton is defined by the sequence $a_{1:K}$ of symbolic decisions. Note that, given the decision rules, such a sequence also defines the sequence of logical state $s_{1:K}$. Practially, in the code, we define a skeleton to be a simple data structure that captures the relevant information from both, $a_{1:K}$ and $s_{1:K}$.
* A skeleton is a list of tuples $(t_\text{start}, t_\text{end}, \text{literal})$, where $t_\text{start}$ denotes the time (actually real-valued phase, but typically the integer k) at which a certain literal becomes true, $t_\text{end}$ denotes the time when the literal was deleted, and `literal` is a some literal, e.g. `(stable hand object)`. In our case, the literals only include symbols (no real-valued constants), the first of which is the predicate, and the others typically refer to objects.
* The skeleton interface to LGP means that you can define a skeleton, and let the algorithm compute a corresponding path. Note that this does not solve the search over symbolic decisions. It allows you to specify the path constraints on the same abstract level that is used internally for LGP.
* Currently implemented skeleton predicates are:

| predicate | arguments | description |
|---|---|---|
| touch | (obj1, obj2) | zero distance equality |
| above | (obj, table) | center of object vertically inside convex table |
| inside | (obj, box) | center of object inside convex box |
| impulse | (obj1, obj2) | impule exchange equation |
| stable | (from, to) | stable but free (7dof) relation |
| stableOn | (from, to) | stable constrained (3dof xyphi) relation |
| dynamic | (from, to) | free dynamics and corresponding equation |
| dynamicOn | (from, to) | constrained 3dof dynamics |
| liftDownUp | | |
| push | | |
| graspSlide| | |

* Generally, a skeleton is translated to a set of objectives (and kinematic switches, and other internal things) for a path or configuration optimization problem. Note that this translation is not unique: In particular, for the same skeleton we can compute different *bounds*, each of which corresponds to a different optimization problem. In `bounds.h` in the `LGP/` module several mappings from a skeleton to an optimization problem are defined. These differ in terms of the objective set, but in terms of parameters like the resolution of the path discretization.
* To summarize, in LGP a decision sequence $a_{1:K}$ maps to a skeleton $S$ -- this mapping is defined by the decision rules which state which literals are created by which decisions (typically defined in `fol.g`). And then the skeleton $S$ is mapped to a constrained optimization problem (as set of objectives) -- this mapping is defined by the bound.

In [3]:
import sys
sys.path.append('../ry')
from libry import *
from numpy import *

In [4]:
K = Config()
K.addFile("../test/boxProblem.g")

In [5]:
komo = K.komo_path(5., 10, .2)

obj = 'ballR'

# this is all yet 'magic' -> clearer interface
komo.addTimeOptimization();
komo.makeObjectsFree([obj]);
komo.addObjective(type=OT.sos, feature=FS.accumulatedCollisions)

S = [
    [1,5], ["dynamicTrans", obj],
    [2,2], ["bounce", "boxBo", obj],
    [3,3], ["bounce", "boxBo", obj],
    [4,4], ["bounce", "boxBo", obj],
    [5,5], ["touch", "target", obj] ]

komo.addSkeleton(S)

In [6]:
komo.optimize();

In [7]:
komoView = komo.view()

In [8]:
komoView=0
komo=0
D=0
K=0

In [7]:
K = Config()
K.addFile("../test/lgp-example.g")

We're creating the same skeleton that'd be created by the decision sequence
```
(grasp baxterR stick) (handover baxterR stick baxterL) (hitSlide stickTip redBall table1) (graspSlide baxterR redBall table1)
```
which is a standard demo in the RSS'18 paper

In [8]:
komo = K.komo_path(4.5, 10, 10.)
komo.addObjective(type=OT.sos, feature=FS.transAccelerations, scale=[1])

In [9]:
S = [
#(grasp baxterR stick)
    [1,1], ["touch", "baxterR", "stick"],
    [1,1], ["stable", "baxterR", "stick"],
    [1,1], ["liftDownUp", "baxterR"],
#(handover baxterR stick baxterL)
    [2,2], ["touch", "baxterL", "stick"],
    [2,4], ["stable", "baxterL", "stick"],
#(hitSlide stickTip redBall table1)
    [3,3], ["touch", "stickTip", "redBall"],
    [3,3], ["impulse", "stickTip", "redBall"],
    [3,3], ["dynamicOn", "table1", "redBall"],
#(graspSlide baxterR redBall table1)
    [4,4], ["graspSlide", "baxterR", "redBall", "table1"] ]

In [10]:
komo.addSkeletonBound(S, BT.path)

In [11]:
komo.optimize()

In [12]:
komoView = komo.view()

In [13]:
komoView=0
komo=0
D=0
K=0